## Alignement des chaînes OCR avec les outils ISRI

In [1]:
import isri_tools

a = "👉 étagères𐌰"
b = "etag.µbleble"

Fonction d'alignement des chaînes.

In [2]:

A, B = isri_tools.align(a, b, 'x')
print(A)
print(B)

👉 étagèrxxes𐌰x
exxtag.µbleble


Pour obtenir la transformation d'alignment.

On obtient ici deux listes de la même taille que les chaînes d'origine et qui donnent les positions dans l'alignement final.

In [3]:
A, B = isri_tools.get_align_map(a, b)
print(A)
print(B)

[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12]
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


## Retagging d'une chaîne OCR à partir des Entités Nommées d'une chaîne XML

In [4]:
from ner_sync_ref import add_tags_prediction

In [5]:
A = "<PER>Anthony</PER>, fab. du <ACT>pêche</ACT>, <LOC>Châtelet</LOC>"
B = "Mme Antoine, fab la pecheur du chatelet et du Faub. St Antoine"
add_tags_prediction(A, B)

'Mme <PER>Antoine</PER>, fab la <ACT>peche</ACT>ur du <LOC>chatelet</LOC> et du Faub. St Antoine'

## Calcul de l'accuracy et rapport

In [6]:
stats = isri_tools.compute_accurary_stats(A, B)
#stats

UnsupportedOperation: fileno